# Data Load and Strucrue Analysis

### RAW 데이터(info.dat)로드 및 형식 확인

In [34]:
# !pip install pandas
import os
import pandas as pd
import json

# 지정된 경로에 있는 폴더명(곡)을 리스트로 반환
levels_path = "./data/CustomLevels/"
all_songs = os.listdir(levels_path)

# 각 폴더에 있는 info.dat 파일들을 load하여 하나의 DataFrame으로 변환
all_info = []
for i, song in enumerate(all_songs):
    try:
        info_path = levels_path+song+"/info.dat"
        with open(info_path, "r", encoding="utf-8") as f:
            loaded_dict = json.load(f)
            loaded_dict.update({'info_path':info_path, 'song_no':i}) # info.dat 파일의 경로와 로드된 순서대로 번호 컬럼을 추가.
            all_info.append(loaded_dict)
    except FileNotFoundError:
        print(f"오류: 파일 경로를 확인해주세요: {info_path}")
    except json.JSONDecodeError as e:
        print(f"오류: JSON 구문 분석 중 오류 발생. 파일 내용이 올바른 JSON 형식이 아닐 수 있습니다. ({e})")

df_info = pd.DataFrame(all_info)
print(df_info.info())
print(df_info.describe())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   _version                       375 non-null    object 
 1   _songName                      375 non-null    object 
 2   _songSubName                   375 non-null    object 
 3   _songAuthorName                375 non-null    object 
 4   _levelAuthorName               375 non-null    object 
 5   _beatsPerMinute                375 non-null    float64
 6   _shuffle                       375 non-null    float64
 7   _shufflePeriod                 375 non-null    float64
 8   _previewStartTime              375 non-null    float64
 9   _previewDuration               375 non-null    float64
 10  _songFilename                  375 non-null    object 
 11  _coverImageFilename            375 non-null    object 
 12  _environmentName               375 non-null    obj

### info.dat 데이터 전처리 (중복 컬럼 합병 및 불필요한 컬럼 제거)

In [35]:
# 확인결과 형식이 다른 샘플은 하나로 확인됨. 
# ('_version'이 아니라 'version'과 같이 언더바가 없는 속성을 갖는 행.)
# 형식이 다른 샘플이 하나 뿐이므로 데이터에서 제거하기로 결정.
df_info = df_info.dropna(subset=['_version']) # version을 갖는 경우 = _version 값이 없어서 Nan 값.

# 행이 제거 되었는지 확인
print(df_info.info())

# 필요한 열만 남기기
columns_to_keep = ['_songName', '_beatsPerMinute', '_songTimeOffset',
                   '_difficultyBeatmapSets', 'info_path', 'song_no']
df_info_extracted = df_info[columns_to_keep]
print(df_info_extracted.head(5))


<class 'pandas.core.frame.DataFrame'>
Index: 375 entries, 0 to 375
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   _version                       375 non-null    object 
 1   _songName                      375 non-null    object 
 2   _songSubName                   375 non-null    object 
 3   _songAuthorName                375 non-null    object 
 4   _levelAuthorName               375 non-null    object 
 5   _beatsPerMinute                375 non-null    float64
 6   _shuffle                       375 non-null    float64
 7   _shufflePeriod                 375 non-null    float64
 8   _previewStartTime              375 non-null    float64
 9   _previewDuration               375 non-null    float64
 10  _songFilename                  375 non-null    object 
 11  _coverImageFilename            375 non-null    object 
 12  _environmentName               375 non-null    object 


### 통합된 info를 .json으로 저장

In [37]:
df_info_extracted.to_json('./data/output_records.json', orient='records', indent=4, force_ascii=False)

pandas 등 라이브러리의 함수 사용법 및 문법에 대해서 구글 AI검색과 Gemini를 활용하였습니다.